In [1]:
!ls ../data

2048.tif  n27_w111_1arc_v3.dted  n37_w107_1arc_v3.tif
512.tif   n37_w104_1arc_v3.tif	 n37_w108_1arc_v3.tif


In [8]:
from osgeo import ogr, osr, gdal

filepath = '../data/n37_w104_1arc_v3.tif'

dataset = gdal.Open(filepath)

# Get the projection information as a WKT (Well-Known Text) string
wkt_projection = dataset.GetProjection()

# Create a SpatialReference object from the WKT string
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(wkt_projection)

0

In [13]:
spatial_ref.Validate()

0

In [15]:
import math
from osgeo import gdal, osr

def scale_distortion(lat, lon, spatial_ref):
    # Convert geographic coordinates to radians
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)

    # Get the partial derivatives of the projection with respect to latitude and longitude
    x_partial_lat, y_partial_lat, x_partial_lon, y_partial_lon = spatial_ref.GetProjParm(osr.SRS_PP_SCALE_FACTOR, 1) * spatial_ref.GetSemiMajor()

    # Calculate the scale distortion at the specified point
    distortion = math.sqrt((x_partial_lat ** 2 + y_partial_lat ** 2 + x_partial_lon ** 2 + y_partial_lon ** 2) / 2)
    return distortion

# Open the GeoTIFF file
dataset = gdal.Open(filepath)

# Get the projection information as a WKT (Well-Known Text) string
wkt_projection = dataset.GetProjection()

# Create a SpatialReference object from the WKT string
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(wkt_projection)

# Specify the latitude and longitude where you want to calculate the distortion
lat, lon = 60, 0

# Calculate and print the scale distortion at the specified point
distortion = scale_distortion(lat, lon, spatial_ref)
print("Scale distortion at (lat, lon) = ({}, {}): {:.4f}".format(lat, lon, distortion))


TypeError: cannot unpack non-iterable float object

In [16]:
!python3 -m pip install pyproj

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 7.8 MB 843 kB/s eta 0:00:01


In [17]:
import math
from osgeo import gdal, osr
import pyproj

# Open the GeoTIFF file
dataset = gdal.Open(filepath)

# Get the projection information as a WKT (Well-Known Text) string
wkt_projection = dataset.GetProjection()

# Create a SpatialReference object from the WKT string
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(wkt_projection)

# Get the EPSG code of the spatial reference
epsg_code = spatial_ref.GetAuthorityCode(None)

# Create pyproj Transformer objects
transformer_to_proj = pyproj.Transformer.from_crs(f"EPSG:{epsg_code}", "EPSG:4326", always_xy=True)
transformer_to_geog = pyproj.Transformer.from_crs("EPSG:4326", f"EPSG:{epsg_code}", always_xy=True)

# Specify the latitude and longitude where you want to calculate the distortion
lat, lon = 60, 0

# Transform the geographic coordinates to projected coordinates
x, y = transformer_to_geog.transform(lon, lat)

# Calculate the scale distortion using the Tissot's indicatrix method
delta = 1e-6
x_plus_delta, y_plus_delta = transformer_to_proj.transform(x + delta, y)
lat_plus_delta, lon_plus_delta = x_plus_delta, y_plus_delta

scale_distortion = math.sqrt((lat_plus_delta - lat)**2 + (lon_plus_delta - lon)**2) / delta

# Print the scale distortion at the specified point
print("Scale distortion at (lat, lon) = ({}, {}): {:.6f}".format(lat, lon, scale_distortion))


Scale distortion at (lat, lon) = (60, 0): 84852813.035279


In [18]:
def scale_distortion(lat, lon, crs_proj):
    # Create a Geod object for distance calculations
    geod = pyproj.Geod(ellps=crs_proj.ellipsoid)

    # Calculate the distance in the projected CRS along latitude and longitude
    delta = 1e-5  # Small delta for derivative approximation
    dist_proj_lat = geod.line_length([(lon, lat), (lon, lat + delta)])
    dist_proj_lon = geod.line_length([(lon, lat), (lon + delta, lat)])

    # Calculate the distance in the geographic CRS (WGS84) along latitude and longitude
    dist_geog_lat = geod.line_length([(lon, lat), (lon, lat + delta)], radians=False)
    dist_geog_lon = geod.line_length([(lon, lat), (lon + delta, lat)], radians=False)

    # Calculate the scale distortion
    scale_distortion_lat = dist_proj_lat / dist_geog_lat
    scale_distortion_lon = dist_proj_lon / dist_geog_lon

    return scale_distortion_lat, scale_distortion_lon

# Open the GeoTIFF file

# Get the projection information as a WKT (Well-Known Text) string
wkt_projection = dataset.GetProjection()

# Create a SpatialReference object from the WKT string
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(wkt_projection)

# Get the EPSG code of the spatial reference
epsg_code = spatial_ref.GetAuthorityCode(None)

# Create the pyproj CRS object for the projected CRS
crs_proj = pyproj.CRS.from_epsg(epsg_code)

# Specify the latitude and longitude where you want to calculate the distortion
lat, lon = 60, 0

# Calculate and print the scale distortion at the specified point
scale_distortion_lat, scale_distortion_lon = scale_distortion(lat, lon, crs_proj)
print("Scale distortion at (lat, lon) = ({}, {}):".format(lat, lon))
print("  Latitude: {:.6f}".format(scale_distortion_lat))
print("  Longitude: {:.6f}".format(scale_distortion_lon))


TypeError: unhashable type: 'pyproj._crs.Ellipsoid'

In [19]:
dataset.GetProjection()

'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'

In [20]:
dataset.SpatialReference()

AttributeError: 'Dataset' object has no attribute 'SpatialReference'

In [21]:
dataset = gdal.Open(filepath)

# Get the projection information as a WKT (Well-Known Text) string
wkt_projection = dataset.GetProjection()

# Create a SpatialReference object from the WKT string
spatial_ref = osr.SpatialReference()
spatial_ref.ImportFromWkt(wkt_projection)

# Extract and print information about the coordinate system and projection
projection_name = spatial_ref.GetAttrValue('PROJECTION')
geographic_coordinate_system = spatial_ref.GetAttrValue('GEOGCS')
datum = spatial_ref.GetAttrValue('DATUM')
unit = spatial_ref.GetAttrValue('UNIT')

print("Projection name: ", projection_name)
print("Geographic coordinate system: ", geographic_coordinate_system)
print("Datum: ", datum)
print("Unit: ", unit)


Projection name:  None
Geographic coordinate system:  WGS 84
Datum:  WGS_1984
Unit:  degree
